# A Machine Learning model that uses MySQL as a DataWarehouse to predict upcoming NFL games

In [3]:
#!jupyter nbconvert --to markdown NFL_Game_Forecasting.ipynb

## The database contains the results and statistics from NFL games dating back to 1967 where each game is represented by two rows - one for each offense/defense matchup. 
## The data was entered into MySQL using the SQL_NFL_set_MySQL.ipynb script also provided

In [55]:
import pandas as pd
import numpy as np
import pymysql as mysql
import joblib
import datetime
import os
import warnings
warnings.filterwarnings("ignore")


## Connect to local MySQL Database

In [56]:
cnx = mysql.connect(
        host='localhost',
        user='root',
        passwd=os.getenv('MYSQL'),
        database='nfl',
        port=int(3306)
)

## Python definitions to execute SQL scripts

In [57]:
cur = cnx.cursor()

def sql(query, params=None):
    cur = cnx.cursor()
    cur.execute(query)
    for x in cur:
        print(x)

def todf(query):
    df = pd.read_sql(query, cnx)
    return df

## The model can be used with user input, so if the provided date is older then there is game data available, the program uses the results of this query to verify that the required data is present, and if it isn’t, the program can change the user input to the value of the ‘oldest_date’.
## Due to MySQL limitations, I couldn’t nest the first query within the second query, so there are two parts.


In [58]:
def oldest_game_date(team, cutoff_date, games_back):

    # PART 1: Get the number of games available for modeling for the specified team
    # I first calculate the count of games for the team present in the database since the cutoff date. Then I subtract the 'games back' variable (amount of prior games required to generate predictions), which leaves only the dates that are suitable for forecasting.
   
    query_count = f"""
        SELECT count(*) AS matches_available
        FROM games
        WHERE game_team = '{team}' 
            AND STR_TO_DATE(game_date, '%Y-%m-%d') >= '{cutoff_date}'
        """

    matches_counts_df = todf(query_count)
    counts = matches_counts_df['matches_available'][0]

    dates_available_for_modeling = (counts - games_back)

    # PART 2: Get the date of the oldest game date available for modeling
    # Uses the 'dates_available_for_modeling' variable to pull the specific value of the ‘oldest_date’ (i.e.  the farthest back I’m able to forecast).
    query_oldestdate = f"""

        SELECT MIN(game_date) AS game_date
        FROM (
            SELECT game_date
            FROM games
            WHERE game_team = '{team}' 
                AND STR_TO_DATE(game_date, '%Y-%m-%d') >= '{cutoff_date}'
            ORDER BY game_date DESC
            LIMIT {dates_available_for_modeling}
        ) AS limited_games

        """

    oldest_dates_df = todf(query_oldestdate)

    # Isolate the date in the last column (oldest)
    oldest_game_date = pd.to_datetime(oldest_dates_df['game_date'][0]).strftime('%Y-%m-%d')

    return oldest_game_date

In [59]:
# for calculations within the pts_model_data_list_function
def weighted_avg(df, col, gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, inte = None):

    gb2 = gb1 + gb2
    gb3 = gb2 + gb3
    gb4 = gb3 + gb4

    average_gb1 = df[col].iloc[:gb1].mean()
    weighted_gb1 = average_gb1 * weight1

    average_gb2 = df[col].iloc[gb1:gb2].mean()
    weighted_gb2 = average_gb2 * weight2


    average_gb3 = df[col].iloc[gb2:gb3].mean()
    weighted_gb3 = average_gb3 * weight3

    average_gb4 = df[col].iloc[gb3:gb4].mean()
    weighted_gb4 = average_gb4 * weight4


    weighted_avg = round(((weighted_gb1 + weighted_gb2 + weighted_gb3 + weighted_gb4) / sum([weight1, weight2, weight3, weight4])), 3)

    
    if inte == 1:
        weighted_avg = int(weighted_avg)

    return weighted_avg

def pts_model_data_list_function(offense_df, defense_df, gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4):  
    
    # Offense

    # field goal percentage last 10 games
    field_goal_percentage = int(round((offense_df['speams_FGM'].iloc[:10].sum() / offense_df['speams_FGA'].iloc[:10].sum()), 3)*100)
    clutch_conversion_percentage = int(round((offense_df['downs_3D_con'].iloc[:20].sum() + offense_df['downs_4D_con'].iloc[:20].sum() + offense_df['speams_2PM'].iloc[:20].sum()) / (offense_df['downs_3D_att'].iloc[:20].sum() + offense_df['downs_4D_att'].iloc[:20].sum() + offense_df['speams_2PA'].iloc[:20].sum()), 3)*100)

    # drives, passing and rushing td/total yards, passing play percentage
    offense_df['drives'] = (offense_df['passing_tds'] + offense_df['rushing_tds'] + offense_df['speams_FGA'] + offense_df['speams_punts_total']).astype(int)
    offense_df['passing_tds_total_yards'] = round((offense_df['passing_tds'] / (offense_df['passing_yds'] + offense_df['rushing_yds']))*10000, 1).astype(int)
    offense_df['rushing_tds_total_yards'] = round((offense_df['rushing_tds'] / (offense_df['passing_yds'] + offense_df['rushing_yds']))*10000, 0).astype(int)
    offense_df['pass_play_percentage'] = round((offense_df['passing_att'] / (offense_df['passing_att'] + offense_df['rushing_att']))*100, 0).astype(int)

    # weighted averages
    rushing_tds_total_yards = weighted_avg(offense_df, 'rushing_tds_total_yards', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    passing_tds_total_yards = weighted_avg(offense_df, 'passing_tds_total_yards', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    pass_play_percentage = weighted_avg(offense_df, 'pass_play_percentage', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    drives = int(weighted_avg(offense_df, 'drives', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*10)
    game_time_off = int(weighted_avg(offense_df, 'game_time_off', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*10)
    penalties = weighted_avg(offense_df, 'game_pen_yds', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    passing_yds = weighted_avg(offense_df, 'passing_yds', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    rushing_yds = weighted_avg(offense_df, 'rushing_yds', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    passing_sacks_total = int(weighted_avg(offense_df, 'passing_sacks_total', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*10)
    passing_int = int(weighted_avg(offense_df, 'passing_int', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*100)
    rushing_fmb = int(weighted_avg(offense_df, 'rushing_fmb', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*100)

    # Defense

    # drives, passing and rushing td/total yards, passing play percentage
    defense_df['drives'] = (defense_df['passing_tds'] + defense_df['rushing_tds'] + defense_df['speams_FGA'] + defense_df['speams_punts_total']).astype(int)
    defense_df['passing_tds_total_yards'] = round((defense_df['passing_tds'] / (defense_df['passing_yds'] + defense_df['rushing_yds']))*10000, 1).astype(int)
    defense_df['rushing_tds_total_yards'] = round((defense_df['rushing_tds'] / (defense_df['passing_yds'] + defense_df['rushing_yds']))*10000, 0).astype(int)
    defense_df['pass_play_percentage'] = round((defense_df['passing_att'] / (defense_df['passing_att'] + defense_df['rushing_att']))*100, 0).astype(int)

    # weighted averages
    def_rushing_tds_total_yards = weighted_avg(defense_df, 'rushing_tds_total_yards', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    def_passing_tds_total_yards = weighted_avg(defense_df, 'passing_tds_total_yards', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    def_drives = int(weighted_avg(defense_df, 'drives', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*10)
    def_game_time_off = int(weighted_avg(defense_df, 'game_time_off', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*10)
    def_passing_yds = weighted_avg(defense_df, 'passing_yds', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    def_rushing_yds = weighted_avg(defense_df, 'rushing_yds', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, 1)
    def_passing_sacks_total = int(weighted_avg(defense_df, 'passing_sacks_total', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*10)
    def_passing_int = int(weighted_avg(defense_df, 'passing_int', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*100)
    def_rushing_fmb = int(weighted_avg(defense_df, 'rushing_fmb', gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)*100)
    def_clutch_conversion_percentage = int(round((defense_df['downs_3D_con'].iloc[:20].sum() + defense_df['downs_4D_con'].iloc[:20].sum() + defense_df['speams_2PM'].iloc[:20].sum()) /
                                            (defense_df['downs_3D_att'].iloc[:20].sum() + defense_df['downs_4D_att'].iloc[:20].sum() + defense_df['speams_2PA'].iloc[:20].sum()), 3)*100)


    pts_model_data_list = [drives, game_time_off, penalties, passing_yds, rushing_yds, 
                       passing_tds_total_yards, rushing_tds_total_yards, pass_play_percentage, field_goal_percentage, 
        clutch_conversion_percentage, passing_sacks_total, passing_int, rushing_fmb, def_passing_yds, def_rushing_yds, 
        def_passing_tds_total_yards, def_rushing_tds_total_yards, def_game_time_off, def_drives,
        def_clutch_conversion_percentage, def_passing_sacks_total, def_passing_int, def_rushing_fmb]
    
    return pts_model_data_list

## When generating forecasts, the program pulls recent game data for each team. This query ensures the program only pulls data relevant to the game at hand. Otherwise, the program will default to loading thousands of useless game details every time it makes a forecast. 

### This query is specialized to focus on one of the two offense/defense matchups within each game (differentiated in the WHERE clause with game_team and game_opp respectively.
### Scripting the {off_date} and {def_date} dates as variables allows the model to be flexible enough to forecast fictional matchups between teams from different years (ie the model can forecast the Minnesota Vikings Offense in 1999 vs the Los Angeles Chargers Defense in 2022).


In [60]:
# avoids nesting the count function in the oldest date function
def match_data_raw(offense, defense, cutoff_date, gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4, off_date = None, def_date = None):

    games_back = gb1 + gb2 + gb3 + gb4 + 1

    if off_date is None:
        off_date = datetime.now().strftime('%Y-%m-%d')  # Default to current date
    if def_date is None:
        def_date = datetime.now().strftime('%Y-%m-%d')  # Default to current date

    # OFF - Date Formatting
    
    oldest_game_off = oldest_game_date(offense, cutoff_date, games_back)

    if off_date < oldest_game_off:
        off_date = oldest_game_off
        print(f"{offense} offense date is too old. Using the oldest available date: {off_date}")
    else:
        pass

    off_query = f"""
        SELECT *
        FROM games
        WHERE game_team = '{offense}'
                AND STR_TO_DATE(game_date, '%Y-%m-%d') <= '{off_date}'
                AND game_result IN ('W', 'L', 'T')
        ORDER BY game_date DESC
        LIMIT {games_back}
        """
    #generate dataframe for offense
    offense_df = todf(off_query)



    # DEF - Date Formatting
    oldest_game_def = oldest_game_date(defense, cutoff_date, games_back)

    if def_date < oldest_game_def:
        def_date = oldest_game_def
        print(f"{defense} defense date is too old. Using the oldest available date: {def_date}")
    else:
        pass

    def_query = f"""
            SELECT *
            FROM games
            WHERE game_opp = '{defense}'
                    AND STR_TO_DATE(game_date, '%Y-%m-%d') <= '{def_date}'
                    AND game_result IN ('W', 'L', 'T')
            ORDER BY game_date DESC
            LIMIT {games_back}
            """
    #generate dataframe defense
    defense_df = todf(def_query)


    offense_df = offense_df.drop(offense_df.index[0])
    defense_df = defense_df.drop(defense_df.index[0])


    return pts_model_data_list_function(offense_df, defense_df, gb1, gb2, gb3, gb4, weight1, weight2, weight3, weight4)

In [61]:
def forecast(team1, team2, team1_date, team2_date, cutoff_date, lr_model, scaler, limit, sd, *weights):

    # create empty dataframe which will be used to train both models
    chalk_22_model_df = pd.DataFrame(columns=[ 
        # Offense
        'drives', 'game_time_off', 'penalties',
        'passing_yds', 'rushing_yds', 'passing_tds_total_yards', 'rushing_tds_total_yards',
        'pass_play_percentage', 'field_goal_percentage', 'clutch_conversion_percentage',
        'passing_sacks_total', 'passing_int', 'rushing_fmb',
        
        # Defense
        'def_passing_yds', 'def_rushing_yds', 'def_passing_tds_total_yards', 'def_rushing_tds_total_yards',
        'def_game_time_off', 'def_drives', 
        'def_clutch_conversion_percentage',
        'def_passing_sacks_total', 'def_passing_int', 'def_rushing_fmb'
        ])

    team1_off_list = match_data_raw(team1, team2, cutoff_date, *weights, team1_date, team2_date)
    team1_off_list_df = pd.DataFrame([team1_off_list], columns=chalk_22_model_df.columns)
    team2_off_list = match_data_raw(team2, team1, cutoff_date, *weights, team2_date, team1_date)
    team2_off_list_df = pd.DataFrame([team2_off_list], columns=chalk_22_model_df.columns)
    chalk_22_model_df = pd.concat([chalk_22_model_df, team1_off_list_df, team2_off_list_df], ignore_index=True)
    chalk_22_model_df_scaled   = pd.DataFrame(scaler.transform(chalk_22_model_df), columns=chalk_22_model_df.columns, index=chalk_22_model_df.index)
    predictions = lr_model.predict(chalk_22_model_df_scaled)
    team1_mean = predictions[0]
    team2_mean = predictions[1]

    team1_wins = 0
    team2_wins = 0
    n = 0

    while n < limit:
        team1_score = np.random.normal(loc=team1_mean, scale=sd)
        team2_score = np.random.normal(loc=team2_mean, scale=sd)

        if team1_score > team2_score:
            team1_wins = team1_wins + 1
        else:
            team2_wins = team2_wins + 1

        n = n + 1

    team1_win_pct = team1_wins / n

    return team1_mean, team2_mean, team1_win_pct

## Input data and run function (The model and scaler functions are loaded as .pkl files from the local directory)

In [62]:
# unique to each game
team1 = 'LAR'
team2 = 'SFO'
team1_date = '2024-9-22'
team2_date = '2024-9-22'


In [63]:
# custom parameters
weights = [5, 3, 3, 6, .3, .25, .25, .2]
cutoff_date = '1991-06-01'
lr_model = joblib.load('lr_model2.pkl')
scaler = joblib.load('chalk_22_scaler.pkl')
limit = 100000
sd = 8

In [64]:
team1_mean, team2_mean, team1_win_pct = forecast(team1, team2, team1_date, team2_date, cutoff_date, lr_model, scaler, limit, sd, *weights)
team1_mean, team2_mean, team1_win_pct

(20.039397915740075, 26.159247116095834, 0.29325)

## The model gives the Los Angeles Rams a 29.4% chance to beat the San Franscisco 49ers on 9/22/2024